In [1]:
import clingo
from clingox.reify import reify_program

In [2]:
def print_answer_sets(
        program,
        verbose=True,
        num_models=0,
):
    control = clingo.Control(["--project", "-Wnone"])
    control.add("base", [], program)
    control.ground([("base", [])])
    control.configuration.solve.models = num_models
    with control.solve(yield_=True) as handle:
        i = 0
        for model in handle:
            i += 1
            if verbose:
                sorted_model = [str(atom) for atom in model.symbols(shown=True)]
                sorted_model.sort()
                print("Answer set: {{{}}}".format(", ".join(sorted_model)))
        handle.get()
    print(f"Num answer sets: {i}")

In [3]:
num_nodes = 6

In [4]:
# Generate a graph with the given number of nodes
program = f"""
    #const n={num_nodes}.
"""
program += """
    node(1..n).
    { edge(X,Y) } :- node(X), node(Y), X != Y.
    edge(X,Y) :- edge(Y,X).

    #show.
    #show edge(X,Y) : edge(X,Y), X < Y.
"""
# Forbid isolated nodes
program += """
    :- node(X), not edge(X,Y) : node(Y), X != Y.
"""
# Ensure that the graph is connected
program += """
    reachable(1).
    reachable(X) :- node(X), reachable(Y), edge(X,Y).
    :- node(X), not reachable(X).
"""
# Symmetry breaking: order nodes by degree
program += """
    degree(X,D) :- node(X), D = #count { Y : node(Y), edge(X,Y) }.
    :- node(X), node(Y), degree(X,DX), degree(Y,DY),
        X < Y, DX > DY.
"""

In [5]:
# Encode Felsner's planarity condition
# (See, e.g., https://page.math.tu-berlin.de/~scheuch/publ/SAT2023-smsplanar.pdf)
program += """
    order(1..3).
    num(1..n).
    1 { ordering(I,U,N) : num(N) } 1 :- order(I), node(U).
    1 { ordering(I,U,N) : node(U) } 1 :- order(I), num(N).
    triple_to_check(U,V,W) :- node(U), node(V), node(W), edge(U,V),
        U != V, U != W, V != W.
    1 { triple_order(U,V,W,I) : order(I) } 1 :- triple_to_check(U,V,W).
    :- triple_to_check(U,V,W), triple_order(U,V,W,I),
        ordering(I,U,NU),
        ordering(I,V,NV),
        ordering(I,W,NW),
        NU > NW.
    :- triple_to_check(U,V,W), triple_order(U,V,W,I),
        ordering(I,U,NU),
        ordering(I,V,NV),
        ordering(I,W,NW),
        NV > NW.
"""

In [6]:
# 
forbidden_base_program = f"""
    #const n={num_nodes}.
"""
forbidden_base_program += """
    node(1..n).
    { edge(X,Y) } :- node(X), node(Y), X != Y.
    edge(X,Y) :- edge(Y,X).
"""

forbidden_programs = [
    # Example 1: forbid having not exactly 6 edges
    """
    :- 6 { edge(X,Y) : node(X), node(Y), X < Y } 6.
    """,
    # # Example 2: forbid having a triangle
    # """
    # triangle :- edge(X1,X2), edge(X2,X3), edge(X1,X3).
    # :- not triangle.
    # """,
]

In [7]:
reified_program = ""
for i, forbidden_program in enumerate(forbidden_programs):
    reified_symbols = reify_program(
        forbidden_base_program + forbidden_program,
        calculate_sccs=True,
    )
    reified_program += "".join([
        f"fprog({i},{symbol}).\n"
        for symbol in reified_symbols
    ])
    reified_program += f"fprog({i}).\n"

# Modified version of:
# https://github.com/potassco/clingo/blob/master/examples/reify/common/metaD.lp
interpreter_program = """
    % NOTE: assumes that a rule has no more than one head
    
    fprog(I,sum(B,G,T)) :- fprog(I,rule(_,sum(B,G))), T = #sum { W,L : fprog(I,weighted_literal_tuple(B,L,W)) }.
    
    % extract supports of atoms and facts
    
    fprog(I,supp(A,B)) :- fprog(I,rule(     choice(H),B)), fprog(I,atom_tuple(H,A)).
    fprog(I,supp(A,B)) :- fprog(I,rule(disjunction(H),B)), fprog(I,atom_tuple(H,A)).
    
    fprog(I,supp(A)) :- fprog(I,supp(A,_)).
    
    fprog(I,atom(|L|)) :- fprog(I,weighted_literal_tuple(_,L,_)).
    fprog(I,atom(|L|)) :- fprog(I,literal_tuple(_,L)).
    fprog(I,atom( A )) :- fprog(I,atom_tuple(_,A)).
    
    fprog(I,fact(A)) :- fprog(I,rule(disjunction(H),normal(B))),
        fprog(I,atom_tuple(H,A)), not fprog(I,literal_tuple(B,_)).
    
    % generate interpretation
    
    fprog(I,true(atom(A)))                         :- fprog(I,fact(A)).
    fprog(I,true(atom(A))); fprog(I,fail(atom(A))) :- fprog(I,supp(A)), not fprog(I,fact(A)).
                   fprog(I,fail(atom(A)))          :- fprog(I,atom(A)), not fprog(I,supp(A)).
    
    fprog(I,true(normal(B))) :- fprog(I,literal_tuple(B)),
        fprog(I,true(atom(L))) : fprog(I,literal_tuple(B, L)), L > 0;
        fprog(I,fail(atom(L))) : fprog(I,literal_tuple(B,-L)), L > 0.
    fprog(I,fail(normal(B))) :- fprog(I,literal_tuple(B, L)), fprog(I,fail(atom(L))), L > 0.
    fprog(I,fail(normal(B))) :- fprog(I,literal_tuple(B,-L)), fprog(I,true(atom(L))), L > 0.
    
    fprog(I,true(sum(B,G))) :- fprog(I,sum(B,G,T)),
        #sum { W,L : fprog(I,true(atom(L))), fprog(I,weighted_literal_tuple(B, L,W)), L > 0 ;
               W,L : fprog(I,fail(atom(L))), fprog(I,weighted_literal_tuple(B,-L,W)), L > 0 } >= G.
    fprog(I,fail(sum(B,G))) :- fprog(I,sum(B,G,T)),
        #sum { W,L : fprog(I,fail(atom(L))), fprog(I,weighted_literal_tuple(B, L,W)), L > 0 ;
               W,L : fprog(I,true(atom(L))), fprog(I,weighted_literal_tuple(B,-L,W)), L > 0 } >= T-G+1.
    
    % verify supported model properties
    
    fprog(I,bot) :- fprog(I,rule(disjunction(H),B)), fprog(I,true(B)), fprog(I,fail(atom(A))) : fprog(I,atom_tuple(H,A)).
    fprog(I,bot) :- fprog(I,true(atom(A))), fprog(I,fail(B)) : fprog(I,supp(A,B)).
    
    % verify acyclic derivability
    
    fprog(I,internal(C,normal(B))) :- fprog(I,scc(C,A)), fprog(I,supp(A,normal(B))),
        fprog(I,scc(C,A')), fprog(I,literal_tuple(B,A')).
    fprog(I,internal(C,sum(B,G)))  :- fprog(I,scc(C,A)), fprog(I,supp(A,sum(B,G))),
        fprog(I,scc(C,A')), fprog(I,weighted_literal_tuple(B,A',W)).
    
    fprog(I,external(C,normal(B))) :- fprog(I,scc(C,A)), fprog(I,supp(A,normal(B))), not fprog(I,internal(C,normal(B))).
    fprog(I,external(C,sum(B,G)))  :- fprog(I,scc(C,A)), fprog(I,supp(A,sum(B,G))),  not fprog(I,internal(C,sum(B,G))).
    
    fprog(I,steps(C,Z-1)) :- fprog(I,scc(C,_)), Z = { fprog(I,scc(C,A)) : not fprog(I,fact(A)) }.
    
    fprog(I,wait(C,atom(A),0))   :- fprog(I,scc(C,A)), fprog(I,fail(B)) : fprog(I,external(C,B)), fprog(I,supp(A,B)).
    fprog(I,wait(C,normal(B),I)) :- fprog(I,internal(C,normal(B))), fprog(I,fail(normal(B))), fprog(I,steps(C,Z)), I = 0..Z-1.
    fprog(I,wait(C,normal(B),I)) :- fprog(I,internal(C,normal(B))), fprog(I,literal_tuple(B,A)),
        fprog(I,wait(C,atom(A),I)), fprog(I,steps(C,Z)), I < Z.
    fprog(I,wait(C,sum(B,G),I))  :- fprog(I,internal(C,sum(B,G))), fprog(I,steps(C,Z)), I = 0..Z-1, fprog(I,sum(B,G,T)),
        #sum { W,L :   fprog(I,fail(atom(L))),   fprog(I,weighted_literal_tuple(B, L,W)), L > 0, not fprog(I,scc(C,L)) ;
               W,L : fprog(I,wait(C,atom(L),I)), fprog(I,weighted_literal_tuple(B, L,W)), L > 0,     fprog(I,scc(C,L)) ;
               W,L :   fprog(I,true(atom(L))),   fprog(I,weighted_literal_tuple(B,-L,W)), L > 0                        } >= T-G+1.
    fprog(I,wait(C,atom(A),I))   :- fprog(I,wait(C,atom(A),0)), fprog(I,steps(C,Z)), I = 1..Z,
        fprog(I,wait(C,B,I-1)) : fprog(I,supp(A,B)), fprog(I,internal(C,B)).
    
    fprog(I,bot) :- fprog(I,scc(C,A)), fprog(I,true(atom(A))), fprog(I,wait(C,atom(A),Z)), fprog(I,steps(C,Z)).
    
    % saturate interpretations that are not answer sets
    
    fprog(I,true(atom(A))) :- fprog(I,supp(A)), not fprog(I,fact(A)), fprog(I,bot).
    fprog(I,fail(atom(A))) :- fprog(I,supp(A)), not fprog(I,fact(A)), fprog(I,bot).
"""

glue_program = """
    :- not fprog(I,bot), fprog(I).
    fprog(I,bot) :- edge(X,Y), fprog(I,output(edge(X,Y),B)), fprog(I,fail(normal(B))), fprog(I).
    fprog(I,bot) :- not edge(X,Y), fprog(I,output(edge(X,Y),B)), fprog(I,true(normal(B))), fprog(I).
"""

program += reified_program
program += interpreter_program
program += glue_program

In [8]:
print_answer_sets(
    program,
    verbose=True,
    num_models=10,
)

Answer set: {edge(1,2), edge(1,6), edge(2,5), edge(3,4), edge(3,5), edge(4,6)}
Answer set: {edge(1,3), edge(1,6), edge(2,4), edge(2,5), edge(3,5), edge(4,6)}
Answer set: {edge(1,2), edge(1,6), edge(2,3), edge(3,5), edge(4,5), edge(4,6)}
Answer set: {edge(1,6), edge(2,3), edge(2,6), edge(3,5), edge(4,5), edge(4,6)}
Answer set: {edge(1,4), edge(1,6), edge(2,3), edge(2,6), edge(3,5), edge(4,5)}
Answer set: {edge(1,6), edge(2,3), edge(3,5), edge(4,5), edge(4,6), edge(5,6)}
Answer set: {edge(1,6), edge(2,6), edge(3,5), edge(4,5), edge(4,6), edge(5,6)}
Answer set: {edge(1,6), edge(2,6), edge(3,4), edge(3,5), edge(4,6), edge(5,6)}
Answer set: {edge(1,6), edge(2,5), edge(3,4), edge(3,5), edge(4,6), edge(5,6)}
Answer set: {edge(1,6), edge(2,6), edge(3,4), edge(3,5), edge(4,5), edge(5,6)}
Num answer sets: 10
